In [2]:
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, LassoCV, ElasticNet, Lars, Ridge
from sklearn.model_selection import cross_val_score , RepeatedKFold , GridSearchCV
from sklearn.datasets import fetch_california_housing
import sklearn.metrics as metrics
import pandas as pd 
import numpy as np 
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots
import geopandas
import pandas_profiling


# California Housing Dataset

The following is an exploration of the California Housing dataset based on the 1990 US Census. We will be using our analysis of the data to recommend areas for our teammate, Tyler Dabat, to go house hunting. We think North Carolina is too humid for him and he would fit it much better as a California resident.

In [ ]:
housing_df = fetch_california_housing(as_frame=True)['frame'] 

##  Data Exploration

In [ ]:
profile = pandas_profiling.ProfileReport(housing_df)
profile.to_widgets()

As usual pandas profiling is a great way to start exploring the data. Exploring the histograms produced, both MedHouseVal and HouseAge have a data floor that caps the maximum value of the variable. 5 for MedHouseVal and 50 for HouseAge. Most of the plots are greatly skewed indicating that outliers could be present, which is confirmed by looking at extremes values. 

There are no missing values in our dataset.

Interrogating the correlation heatmaps, it seems like MedInc is highly correlated with the MedHouseVal. It also seems that Longitude and Latitude are correlated to MedHouseVal but to a lesser extent.

In [ ]:
plt.figure(figsize=(12,12))
plt.xticks(rotation=60)
sns.heatmap(housing_df.corr(),linewidths=0.2, cmap="RdBu", annot=True)

This correlation matrix also further verifies the linear correlation (0.69) between MedInc and the dependent variable MedHouseVal.

AveRooms and AveBedrms also have a strong correlation (0.85). Additionally, Latitude and Longitude have strong negative correlation (-0.92). This is consistent with the general shape of California being similar to a line in the form of $y = -x$ 

In [ ]:
fig = subplots.make_subplots(rows=3,cols=3)

for i in range(9):
    row=i//3 +1 
    col=i%3 +1
    fig.append_trace(go.Violin(
        y=housing_df.iloc[:,i],name=housing_df.columns[i]
    )
    ,row=row,col=col)
fig.update_traces(box_visible=True,
                  jitter=0.05,  meanline_visible=True)
fig.update_layout(width=1000, height=800, title_text= 'Distibutions of our Variables')
fig.show()

From these violin diagrams, it can be found that there are statistically abnormal values occur in almost all of the variables most notably in MedInc, AveRooms, AveBedrms, AveOccup, and Population. We see two bumps in the Latitude and Longitude indicating clustering in where the block groups are (higher population density). A cluster of points corresponding to the floor in MedHouseVal can also be seen.

In [ ]:
fig = px.scatter_matrix(
    housing_df,
    dimensions = housing_df.columns[:-1],
    labels = {housing_df.columns[i] for i in range(len(housing_df.columns))},
    color="MedHouseVal",
    opacity = 0.5
)
fig.update_layout(
    title = "California Housing Dataset Scatter Matrix",
    height= 1000,
    width=1000
)
fig.show()

Looking at the scatter matrix the latitude and longitude have two spikes in their corresponding plots vs. Medium House Value. These are most likely due to cities with high housing values.

## GIS Plots

Exploratory GIS plots are shown below. These should give us a better geographical idea of the houing price distribution in California.

In [ ]:
house_geo_df = geopandas.GeoDataFrame(housing_df, 
                                      geometry=geopandas.points_from_xy(housing_df.Longitude, 
                                                                        housing_df.Latitude))

In [ ]:
housing_df['MedInc'] = housing_df['MedInc'] * 10000
housing_df['MedHouseVal'] = housing_df['MedHouseVal'] * 100000
fig = px.scatter_mapbox(housing_df, lat= "Latitude", lon="Longitude",
                size = "MedInc", 
                color ="MedHouseVal", 
                size_max = 10, 
                color_continuous_scale = px.colors.sequential.Plasma,
                center={"lat": 37.5, "lon": -120},
                zoom=5.5,
                mapbox_style="carto-darkmatter")
fig.update_traces(hovertemplate="Median House Value : %{marker.color:$,.0f}<br>" + 
                                "Median Income : %{marker.size:$,.0f}")
fig.update_layout(
                title_text='Map',
                margin={"r":10,"t":0,"l":10,"b":-0},
                width=990,
                height=900)
fig.show()

What is most noticeable about the plot above is the higer prices centered around the Los Angeles and the San Fransisco area.

### Shape File

We brought in a shape file to look at the data in terms of the blcok groups. Depicted below is the shape file showing all of the California block groups in th 1990 census data.

In [ ]:
calif_shape = geopandas.read_file('bg06_d90.shp')
fig, ax = plt.subplots(1, 1, figsize = (15,15))
calif_shape.plot(ax=ax)
plt.show()

In [ ]:
merged_df = geopandas.sjoin(calif_shape, house_geo_df, how="left", op='intersects')

In [ ]:
fig = px.choropleth_mapbox(merged_df, geojson=merged_df.geometry, 
                    locations=merged_df.index,
                    color='MedHouseVal',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    center={"lat": 37.5, "lon": -120},
                    zoom=5.5,
                    mapbox_style="carto-darkmatter",
                    custom_data = ['MedInc', 'BG06_D90_I'])
fig.update_traces(hovertemplate="<b>Block Group # %{customdata[1]}</b><br><br>"+ 
                                "<b style='color:red;'>Median House Value</b> : %{z:$,.0f}<br>" + 
                                "Median Income : %{customdata[0]:$,.0f}"
                              )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(
                title_text='Map',
                margin={"r":10,"t":0,"l":10,"b":-0},
                width=990,
                height=900)
fig.show()

#### County View

In [ ]:
county_df = merged_df.dissolve(by='CO', aggfunc='mean')
county_df['CO'] = county_df.index
fig = px.choropleth_mapbox(county_df, geojson=county_df.geometry, 
                    locations=county_df.index,
                    color='MedHouseVal',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    center={"lat": 37.5, "lon": -120},
                    zoom=5.5,
                    mapbox_style="carto-darkmatter",
                    custom_data = ['MedInc', 'CO'])
fig.update_traces(hovertemplate="<b>County # %{customdata[1]}</b><br><br>"+ 
                                "<b style='color:red;'>Median House Value</b> : %{z:$,.0f}<br>" + 
                                "Median Income : %{customdata[0]:$,.0f}"
                              )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":10,"t":0,"l":10,"b":-0},
                 width=990,
                 height=900,
                 title_text='Map')
fig.show()

Just looking at the counties tells much of the same story. The southern coastal areas tend to have higher values with San Fransisco noticably being the highest area.

#### Cost Effectiveness

The two main factors in Tyler's decision are going to be his expected income and expected price of housing. Let's look at an "Affordibility Ratio"; the ratio of income to housing prices.

In [ ]:
merged_df['ratio'] = (merged_df.MedInc) / (merged_df.MedHouseVal)
fig = px.choropleth_mapbox(merged_df, geojson=merged_df.geometry, 
                    locations=merged_df.index,
                    color='ratio',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    center={"lat": 37.5, "lon": -120},
                    zoom=5.5,
                    mapbox_style="carto-darkmatter",
                    custom_data = ['MedHouseVal', 'MedInc', 'BG06_D90_I', 'Population'])
fig.update_traces(hovertemplate="<b>Block Group # %{customdata[2]}</b><br><br>"+ 
                                "<b style='color:red;'>Affordability Ratio</b> : %{z:.3f}<br>" + 
                                "Median House Value: %{customdata[0]:$,.0f}<br>"+
                                "Median Income: %{customdata[1]:$,.0f}<br>" +
                                "population: %{customdata[3]}"
                              )
fig.update_layout(
                title_text='Map',
                margin={"r":10,"t":0,"l":10,"b":-0},
                width=990,
                height=900)
fig.show()

Wow! There is an affordable area for Tyler to move to. Even better it looks like it is near Los Angeles. Investigating further, this area is the Angeles National Forest with little housing other then those associated with the National Forest Service. Tyler will have to choose between a career change as a Forest Ranger or staying in North Carolina.

## Regression - The Old Fashioned Way

Visual representations of the data placed in California tells us a lot.  But what can we learn form regression?  Are there quality predictors that could have helped Tyler determine if there is a place for him in California?

As we noticed from the correlation heatmap, Meadian Income has the greatest correlation with Median House Value.  This should be a great place to start.  

In [ ]:
#only run once to set back to normal after display
housing_df['MedInc'] = housing_df['MedInc'] / 10000
housing_df['MedHouseVal'] = housing_df['MedHouseVal'] / 100000

In [ ]:
X = housing_df['MedInc'].values.reshape(-1, 1).copy()
y = housing_df['MedHouseVal'].values
ols = LinearRegression().fit(X, y)
housing_df['ols_pred'] = ols.predict(X)


In [ ]:
def regression_results(y_true, y_pred):
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    try:
        mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    except:
        mean_squared_log_error = -9999
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error , 4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse) , 4))
    
regression_results(y, housing_df['ols_pred'])

In [ ]:
plt.scatter(X , y , color='red')
plt.plot(X , ols.predict(X) , color='green')
plt.title('Median Income vs Mdeian House Value')
plt.xlabel('Median Income')
plt.ylabel('Median House Value')
plt.show()

### How did we do?  

This model fails to explain more than .5 of the variance, let alone something more desirable > .8.  This single predictor is not great.  The almost solid line of points at 5 (Median Hosue Value) looks like it is hurting the model.  Lets try removing those values as Tyler cannot afford those homes anyway. The line is created from a behavior where all homes greater than 500,000 are set to 500,001.  Removing these homes hopefully improves our model.  

In [ ]:
new_housing_df = housing_df[housing_df['MedHouseVal'] < 5].copy()
newX = new_housing_df['MedInc'].values.reshape(-1, 1).copy()
newy = new_housing_df['MedHouseVal'].values
newOls = LinearRegression().fit(newX, newy)
new_housing_df['new_ols_pred'] = newOls.predict(newX)

regression_results(newy, new_housing_df['new_ols_pred'])

In [ ]:
plt.scatter(newX , newy , color='red')
plt.plot(newX , newOls.predict(newX) , color='green')
plt.title('Median Income vs Mdeian House Value')
plt.xlabel('Median Income')
plt.ylabel('Median House Value')
plt.show()

### Is There more worth doing with OLS?

Unfortunately removing the homes with innacurate median values didn't improve our model.  It in fact slightly decreaed the effectiveness of our model.  That being said, all that was used to predict Median House Value was Median Income.  Since there is no risk in overfitting in the context of minning.  Lets see how a model that uses all predictors performs.  

In [ ]:
allX = new_housing_df[['MedInc' , 'AveRooms' , 'HouseAge' , 'AveBedrms' , 'Population' , 'AveOccup' , 'Latitude' , 'Longitude']]
ally = new_housing_df['MedHouseVal'].values
allOls = LinearRegression().fit(allX, ally)

new_housing_df['all_ols_pred'] = allOls.predict(allX)
regression_results(ally, new_housing_df['all_ols_pred'])

### Slight Improvement - Now for Lasso Regression

Using all of the predictors has improved our model performance slightly.  But can we do better?  Lets see what Lasso Regression can do for us.

In [ ]:
lasso_model = Lasso(alpha=1)
cross_val = RepeatedKFold(n_splits=10 , n_repeats=3 , random_state=1)
scores = cross_val_score(lasso_model , allX , ally, scoring='neg_mean_absolute_error' , cv=cross_val , n_jobs=-1)
scores = np.absolute(scores)
print('Initial Evaluation Mean MAE: %.3f (%.3f)' %(np.mean(scores), np.std(scores)))
lasso_model.fit(allX , ally)
new_housing_df['van_lasso'] = lasso_model.predict(allX)
regression_results(ally , new_housing_df['van_lasso'])

### This Lasso Model Needs Work

Vanilla Lasso regression does not produce a very appealing model.  We need to work on optimizing this model.  Lets tune the hyperparameter alpha and see which predictors are best.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

cross_val = RepeatedKFold(n_splits=10 , n_repeats=3 , random_state=1)
tun_model = LassoCV(alphas=np.arange(0,1,0.0001) , cv=cross_val, n_jobs=-1)
tun_model.fit(allX,ally)
print('Tuned alpha %f' %tun_model.alpha_)


We obtained an alpha of 0.0015, with a lot of warnings about non-convergence.   With a penalty this low, we shouldn't expect much difference in the lasso regression from regular Multiple Regression. (I would like to run more iterations, but it crashes my kernel)  Lets see how the model does.

In [ ]:
new_housing_df['tun_lasso'] = tun_model.predict(allX)
regression_results(ally , new_housing_df['tun_lasso'])
print('Coefs: %s' % tun_model.coef_)

It looks as though our best efforts in creating a model that out performs OLS have not produced anything worthy.  Lets try a few other regression techniques and see what we get.  We will continue to use all predictors as none of them were penalized out in the Lasso regression.

### Elastic Net

In [ ]:
elastic = ElasticNet(alpha=.01)
elastic.fit(allX, ally)
new_housing_df['elastic'] = elastic.predict(allX)
regression_results(ally , new_housing_df['elastic'])

### Lars

In [ ]:
lars = Lars().fit(allX, ally)
new_housing_df['lars'] = lars.predict(allX)
regression_results(ally , new_housing_df['lars'])

### Ridge

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
ridge = RidgeCV(alphas=(0,1,0.01), cv=cv, scoring='neg_mean_absolute_error')
ridge.fit(allX, ally)
new_housing_df['ridge'] = ridge.predict(allX)
regression_results(ally , new_housing_df['ridge'])
print('alpha: %s' % ridge.alpha_)

# Conclusion

As it turns out, creating a model that explains most of the variance is not in the cards for Tyler.  Optimized Ridge and Lasso don't perform much better than regular multiple regression.  Unfortunately for Tyler, this does not increase his ability to optimize his choice of housing in California.  It looks like he has become a Park Ranger in the Angeles National Forest, or live in the forrest(If they allow that) to move to his dream state of California.